In [70]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)

def distancia(p1, p2):
    """Calcula la distancia euclidiana entre dos puntos."""
    return np.linalg.norm(np.array(p1) - np.array(p2))
escalate_value = 0
rotate_value=0
h=0
w=0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame= cv2.flip(frame, 1)
    # Convertir imagen a RGB (MediaPipe usa RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # Variables para guardar coordenadas
    left_index = None
    right_index = None

    cx, cy = w // 2, h // 2
    
    rect = ((cx, cy), (escalate_value, escalate_value), rotate_value)
    box = cv2.boxPoints(rect)
    box = np.int32(box)
    cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)

    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            label = handedness.classification[0].label  # 'Left' o 'Right'
            #print(label)
            h, w, _ = frame.shape

            
            # Coordenadas del índice (landmark 8)
            index_tip = hand_landmarks.landmark[8]
            x, y = int(index_tip.x * w), int(index_tip.y * h)
            
            wrist = hand_landmarks.landmark[0]
            x1, y1 = int(wrist.x * w), int(wrist.y * h)
            
            pinky_tip = hand_landmarks.landmark[20]
            x2, y2 = int(pinky_tip.x * w), int(pinky_tip.y * h)
            
            # Guardar según la mano
            if label == 'Left':
                left_index = (x, y)
                left_wrist = (x1, y1)
                left_pinky = (x2, y2)
                
            elif label == 'Right':
                right_index = (x, y)
                right_wrist = (x1, y1)
                right_pinky = (x2, y2)

            # Dibujar los landmarks (opcional)
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            

        # Si ambas manos detectadas, dibujar línea entre los dos puntos
        if left_index and right_index:
            #cv2.line(frame, left_index, right_index, (0, 255, 0), 3)
            cv2.circle(frame, left_index, 8, (255, 0, 0), -1)
            cv2.circle(frame, right_index, 8, (0, 0, 255), -1)
            escalate_value = distancia(left_index, right_index)//2
            rotate_value = (right_index[1] - left_index[1])*0.2

        elif right_wrist and right_index and right_pinky:
            cv2.circle(frame, right_index, 8, (0, 0, 255), -1)
            cv2.circle(frame, right_wrist, 8, (0, 0, 255), -1)
            escalate_value = distancia(right_index, right_wrist)
            rotate_value = (right_index[1] - right_pinky[1])*0.5

        elif left_wrist and left_index and left_pinky:
            cv2.circle(frame, left_index, 8, (0, 0, 255), -1)
            cv2.circle(frame, left_wrist, 8, (0, 0, 255), -1)
            escalate_value = distancia(left_index, left_wrist)
            rotate_value = (left_index[1] - left_pinky[1])*0.5
        
            

    cv2.imshow("Line", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()